In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
from keras.layers import LSTM, Dropout, Dense, Embedding
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [2]:
parent_path = 'FakeNewsDetector/dataset/fake-news-FromKaggle/'
model= load_model('m1.h5')

2022-09-26 19:22:52.414628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def data_process(dataframe):
    df = dataframe
    df = df.drop(columns=['id', 'title', 'author'], axis=1)
    df = df.dropna(axis=0)
    df['clean_news'] = df['text'].str.lower()
    df['clean_news'] = df['clean_news'].str.replace('[^A-Za-z0-9\s]', '')
    df['clean_news'] = df['clean_news'].str.replace('\n', '')
    df['clean_news'] = df['clean_news'].str.replace('\s+', ' ')

    # delete stopwords and punctuation
    stop = stopwords.words('english')
    df['clean_news'] = df['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))

    return df

## First Test

In [4]:
df_test = pd.read_csv(parent_path + 'test.csv')
df_label = pd.read_csv(parent_path + 'submit.csv')
df_test = pd.merge(df_label, df_test, on='id')
df_test = data_process(df_test)

/var/folders/yn/tcg080tn52v8gfv4d82tm89h0000gn/T/ipykernel_28595/583088388.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_news'] = df['clean_news'].str.replace('[^A-Za-z0-9\s]', '')
/var/folders/yn/tcg080tn52v8gfv4d82tm89h0000gn/T/ipykernel_28595/583088388.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_news'] = df['clean_news'].str.replace('\s+', ' ')


In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_test['clean_news'])
sequences = tokenizer.texts_to_sequences(df_test['clean_news'])
padded_seq = pad_sequences(sequences, maxlen=500, padding='post', truncating='post')

In [7]:
# prediction
result = model.predict(padded_seq)
result_classify = np.round(result).astype(int)

163/163 [==============================] - 15s 90ms/step


In [8]:
# calculate accuracy
np.sum((np.array(df_test['label']) == result_classify.T.squeeze()) == 1) / len(result)

0.8401694588869633

## Second Test

In [9]:
# data from other dataset
# data in this dataset are all political fake news
df_test2 = pd.read_csv('FakeNewsNet_Politifact.csv')
df_test2.head(2)

,content,title,publish_date,url,news_id,label
0,"On Sept. 28, pastors from 20 states will give ...",Pastors To Protest IRS Rules on Political Advo...,2008-09-19,http://www.pewforum.org/2008/09/19/pastors-to-...,/Users/zch/Desktop/DS440/dataset/FakeNewsNet-m...,fake
1,"UNHCR, the UN Refugee Agency, has welcomed new...","UN Refugee Agency welcomes arrival of 10,000th...",2016-09-05,http://www.unhcr.org/en-us/news/press/2016/9/5...,/Users/zch/Desktop/DS440/dataset/FakeNewsNet-m...,fake


In [11]:
df_test2 = df_test2.dropna(axis=0)
df_test2['clean_news'] = df_test2['content'].str.lower()
df_test2['clean_news'] = df_test2['clean_news'].str.replace('[^A-Za-z0-9\s]','')
df_test2['clean_news'] = df_test2['clean_news'].str.replace('\n','')
df_test2['clean_news'] = df_test2['clean_news'].str.replace('\s+',' ')

/var/folders/yn/tcg080tn52v8gfv4d82tm89h0000gn/T/ipykernel_28595/1301744868.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test2['clean_news'] = df_test2['clean_news'].str.replace('[^A-Za-z0-9\s]','')
/var/folders/yn/tcg080tn52v8gfv4d82tm89h0000gn/T/ipykernel_28595/1301744868.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test2['clean_news'] = df_test2['clean_news'].str.replace('\s+',' ')


In [12]:
stop = stopwords.words('english')
df_test2['clean_news'] = df_test2['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))
df_test2.head(2)

,content,title,publish_date,url,news_id,label,clean_news
0,"On Sept. 28, pastors from 20 states will give ...",Pastors To Protest IRS Rules on Political Advo...,2008-09-19,http://www.pewforum.org/2008/09/19/pastors-to-...,/Users/zch/Desktop/DS440/dataset/FakeNewsNet-m...,fake,sept 28 pastors 20 states give politically bas...
1,"UNHCR, the UN Refugee Agency, has welcomed new...","UN Refugee Agency welcomes arrival of 10,000th...",2016-09-05,http://www.unhcr.org/en-us/news/press/2016/9/5...,/Users/zch/Desktop/DS440/dataset/FakeNewsNet-m...,fake,unhcr un refugee agency welcomed news arrival ...


In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_test2['clean_news'])

sequences = tokenizer.texts_to_sequences(df_test2['clean_news'])
padded_seq = pad_sequences(sequences, maxlen=500, padding='post', truncating='post')

In [15]:
# prediction
result = model.predict(padded_seq)
result_classify = np.round(result).astype(int)

9/9 [==============================] - 1s 92ms/step


In [19]:
df_test2[df_test2['label']!='fake']

,content,title,publish_date,url,news_id,label,clean_news


In [16]:
np.sum(result_classify)/len(result_classify)

0.8483754512635379